In [4]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
Mapa Interativo: Uso do Solo + Valores do Solo - Londrina
Camada 1: Classes de uso do solo (patches 224x224 pixels)
Camada 2: Valores do solo preditos (quadrados 109.45m × 109.45m)
"""

import pandas as pd
import numpy as np
import folium
from folium import plugins
from shapely.geometry import Polygon
from pyproj import Transformer
import matplotlib.colors as mcolors

# ============================================================================
# 1. DEFINIÇÃO DE CORES E PALETAS
# ============================================================================

# Cores para classes de uso do solo
class_colors = {
    'bare': '#bf8040',        # Marrom (solo exposto)
    'bush': '#009900',        # Verde médio
    'crop': '#ffdf99',        # Amarelo claro (agricultura)
    'grass': '#33ff33',       # Verde claro/lima
    'hduf': '#69000d',        # Vermelho escuro (alta densidade urbana)
    'industrial': '#e93529',  # Vermelho alaranjado (industrial)
    'lduf': '#c7171c',        # Vermelho médio (baixa densidade urbana)
    'mduf': '#a10e15',        # Vermelho médio-escuro (média densidade urbana)
    'tree': '#003300',        # Verde escuro
    'water': '#0b9fd5'        # Azul (água)
}

# Paleta tim.colors() do R para valores do solo
def tim_colors(n=1000):
    """Recria a paleta tim.colors() do R"""
    colors = [
        '#000080',  # azul marinho
        '#0000CC',  # azul escuro
        '#0040FF',  # azul
        '#0080FF',  # azul claro
        '#00BFFF',  # azul céu
        '#00FFFF',  # ciano
        '#00FFBF',  # ciano-verde
        '#00FF80',  # verde-ciano
        '#00FF40',  # verde claro
        '#00FF00',  # verde
        '#40FF00',  # verde-amarelo
        '#80FF00',  # amarelo-verde
        '#BFFF00',  # amarelo claro
        '#FFFF00',  # amarelo
        '#FFBF00',  # amarelo-laranja
        '#FF8000',  # laranja
        '#FF4000',  # laranja-vermelho
        '#FF0000'   # vermelho
    ]
    
    cmap = mcolors.LinearSegmentedColormap.from_list("tim", colors, N=n)
    return [mcolors.rgb2hex(cmap(i)) for i in np.linspace(0, 1, n)]

pal_valores = tim_colors(1000)

# ============================================================================
# 2. FUNÇÕES AUXILIARES
# ============================================================================

def parse_coordinates(coord_str):
    """Extrai lat, lon de string 'lat,lon'"""
    lat, lon = map(float, coord_str.split(','))
    return lat, lon

def create_patch_polygon(row):
    """Cria polígono do patch a partir dos 4 vértices"""
    tl_lat, tl_lon = parse_coordinates(row['top_left'])
    tr_lat, tr_lon = parse_coordinates(row['top_right'])
    bl_lat, bl_lon = parse_coordinates(row['bottom_left'])
    br_lat, br_lon = parse_coordinates(row['bottom_right'])
    
    # Folium usa [lat, lon]
    return [
        [tl_lat, tl_lon],  # top_left
        [tr_lat, tr_lon],  # top_right
        [br_lat, br_lon],  # bottom_right
        [bl_lat, bl_lon],  # bottom_left
        [tl_lat, tl_lon]   # fecha o polígono
    ]

def create_square_utm(utm_x, utm_y, half_size):
    """Cria um quadrado em coordenadas UTM"""
    return Polygon([
        (utm_x - half_size, utm_y - half_size),  # inferior esquerdo
        (utm_x + half_size, utm_y - half_size),  # inferior direito
        (utm_x + half_size, utm_y + half_size),  # superior direito
        (utm_x - half_size, utm_y + half_size),  # superior esquerdo
        (utm_x - half_size, utm_y - half_size)   # fecha o polígono
    ])

def transform_polygon_to_latlon(polygon_utm, transformer):
    """Converte polígono UTM para lat/lon"""
    coords_utm = list(polygon_utm.exterior.coords)
    coords_latlon = []
    
    for x_utm, y_utm in coords_utm:
        lon, lat = transformer.transform(x_utm, y_utm)
        coords_latlon.append([lat, lon])  # Folium usa [lat, lon]
    
    return coords_latlon

# ============================================================================
# 3. CARREGAR DADOS E CALCULAR ÁREAS
# ============================================================================

print("Carregando dados...")


# ============================================================================
# FUNÇÕES PARA CÁLCULO DE ÁREAS
# ============================================================================

def process_landcover_csv(csv_path, city_name):
    """
    Processa CSV de cobertura do solo e retorna GeoDataFrame com geometrias de polígonos
    """
    from shapely.geometry import Point, Polygon
    import geopandas as gpd
    
    # Carregar o arquivo CSV
    df = pd.read_csv(csv_path, sep=";")
    
    # Extrair apenas o nome do arquivo
    df["name"] = df["name"].str.split("/").str[-1]
    
    # Criar o DataFrame para as janelas
    df_janelas = df.copy()
    
    # Converter as coordenadas dos cantos para objetos Point
    df_janelas["top_left"] = df_janelas["top_left"].apply(lambda x: Point(float(x.split(",")[1]), float(x.split(",")[0])))
    df_janelas["bottom_left"] = df_janelas["bottom_left"].apply(lambda x: Point(float(x.split(",")[1]), float(x.split(",")[0])))
    df_janelas["top_right"] = df_janelas["top_right"].apply(lambda x: Point(float(x.split(",")[1]), float(x.split(",")[0])))
    df_janelas["bottom_right"] = df_janelas["bottom_right"].apply(lambda x: Point(float(x.split(",")[1]), float(x.split(",")[0])))
    
    # Criar a geometria de polígonos
    df_janelas["geometry"] = df_janelas.apply(lambda row: Polygon([(row["top_left"].x, row["top_left"].y),
                                                                   (row["bottom_left"].x, row["bottom_left"].y),
                                                                   (row["bottom_right"].x, row["bottom_right"].y),
                                                                   (row["top_right"].x, row["top_right"].y)]), axis=1)
    
    # Criar GeoDataFrame
    gdf_squares = gpd.GeoDataFrame(df_janelas, geometry="geometry")
    gdf_squares.crs = "epsg:4326"
    gdf_squares_UTM = gdf_squares.to_crs("epsg:29192")
    
    # Criar coluna pred
    class_to_number = {"bare": 0, "bush": 1, "crop": 2, "grass": 3, "hduf": 4,
                      "industrial": 5, "lduf": 6, "mduf": 7, "tree": 8, "water": 9}
    gdf_squares_UTM["pred"] = gdf_squares_UTM["predicted_class"].map(class_to_number)
    gdf_squares_UTM["municipality"] = city_name
    
    return gdf_squares_UTM

def calculate_areas(gdf, area_pixel_m2, city_name):
    """Calcula áreas de cobertura do solo por classe"""
    class_names = ["bare", "bush", "crop", "grass", "hduf", "industrial", "lduf", "mduf", "tree", "water"]
    results = {}
    for class_index, class_name in enumerate(class_names):
        class_points = gdf[gdf["pred"] == class_index].shape[0]
        area_m2 = class_points * area_pixel_m2
        area_km2 = area_m2 / 1e6
        results[f"area_{class_name}_km2_{city_name}"] = area_km2
    return results

# ============================================================================
# 3. CARREGAR DADOS E CALCULAR ÁREAS
# ============================================================================

# Dados de uso do solo de Londrina, Ibiporã e Cambé
df_uso_lda = pd.read_csv('/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_londrina/inference_results_with_positions_20250609_083252.csv', sep=';')
print(f"✓ Uso do solo: {len(df_uso_lda):,} patches")

df_uso_ibip = pd.read_csv('/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_ibipora/inference_results_with_positions_20250609_164529.csv', sep=';')
print(f"✓ Uso do solo: {len(df_uso_ibip):,} patches")

df_uso_camb = pd.read_csv('/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_cambe/inference_results_with_positions_20250609_161440.csv', sep=';')
print(f"✓ Uso do solo: {len(df_uso_camb):,} patches")

df_uso_apuc = pd.read_csv('/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_apucarana/inference_results_with_positions_20251118_114239.csv', sep=';')
print(f"✓ Uso do solo: {len(df_uso_apuc):,} patches")

df_uso_arap = pd.read_csv('/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_arapongas/inference_results_with_positions_20251118_121155.csv', sep=';')
print(f"✓ Uso do solo: {len(df_uso_arap):,} patches")

########### NOVOS DATAFRAMES PARA INCLUIR NA TABELA ##################
df_uso_cambir = pd.read_csv('/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_cambira/inference_results_with_positions_20251118_122253.csv', sep=';')
print(f"✓ Uso do solo: {len(df_uso_cambir):,} patches")

df_uso_jand = pd.read_csv('/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_jandaia/inference_results_with_positions_20251118_122910.csv', sep=';')
print(f"✓ Uso do solo: {len(df_uso_jand):,} patches")

df_uso_mand = pd.read_csv('/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_mandaguari/inference_results_with_positions_20251118_125143.csv', sep=';')
print(f"✓ Uso do solo: {len(df_uso_mand):,} patches")

df_uso_mari = pd.read_csv('/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_marialva/inference_results_with_positions_20251118_124329.csv', sep=';')
print(f"✓ Uso do solo: {len(df_uso_mari):,} patches")

df_uso_rol = pd.read_csv('/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_rolandia/inference_results_with_positions_20251118_130921.csv', sep=';')
print(f"✓ Uso do solo: {len(df_uso_rol):,} patches")

df_uso_sar = pd.read_csv('/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_sarandi/inference_results_with_positions_20251118_131430.csv', sep=';')
print(f"✓ Uso do solo: {len(df_uso_sar):,} patches")

df_uso_maring = pd.read_csv('/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_maringa/inference_results_with_positions_20251118_133121.csv', sep=';')
print(f"✓ Uso do solo: {len(df_uso_maring):,} patches")

##########################################################################





# Processar dados de uso do solo para cálculo de áreas
print("\nProcessando dados para cálculo de áreas...")
gdf_lda = process_landcover_csv("/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_londrina/inference_results_with_positions_20250609_083252.csv", "Londrina")
gdf_ibip = process_landcover_csv("/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_ibipora/inference_results_with_positions_20250609_164529.csv", "Ibipora")
gdf_camb = process_landcover_csv("/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_cambe/inference_results_with_positions_20250609_161440.csv", "Cambe")
gdf_apuc = process_landcover_csv("/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_apucarana/inference_results_with_positions_20251118_114239.csv", "Apucarana")
gdf_arap = process_landcover_csv("/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_arapongas/inference_results_with_positions_20251118_121155.csv", "Arapongas")
gdf_cambir = process_landcover_csv("/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_cambira/inference_results_with_positions_20251118_122253.csv", "Cambira")
gdf_jand = process_landcover_csv("/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_jandaia/inference_results_with_positions_20251118_122910.csv", "Jandaia")
gdf_mand = process_landcover_csv("/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_mandaguari/inference_results_with_positions_20251118_125143.csv", "Mandaguari")
gdf_mari = process_landcover_csv("/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_marialva/inference_results_with_positions_20251118_124329.csv", "Marialva")
gdf_rol = process_landcover_csv("/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_rolandia/inference_results_with_positions_20251118_130921.csv", "Rolandia")
gdf_sar = process_landcover_csv("/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_sarandi/inference_results_with_positions_20251118_131430.csv", "Sarandi")
gdf_maring = process_landcover_csv("/Users/fjcosta/Documents/projects/DINO_LORA/modelos/final_models/inference_results_maringa/inference_results_with_positions_20251118_133121.csv", "Maringa")

# Calcular áreas
area_pixel_m2 = 11950.766300078105
results_lda = calculate_areas(gdf_lda, area_pixel_m2, "Londrina")
results_ibip = calculate_areas(gdf_ibip, area_pixel_m2, "Ibipora")
results_camb = calculate_areas(gdf_camb, area_pixel_m2, "Cambe")
results_apuc = calculate_areas(gdf_apuc, area_pixel_m2, "Apucarana")
results_arap = calculate_areas(gdf_arap, area_pixel_m2, "Arapongas")
results_cambir = calculate_areas(gdf_cambir, area_pixel_m2, "Cambira")
results_jand = calculate_areas(gdf_jand, area_pixel_m2, "Jandaia")
results_mand = calculate_areas(gdf_mand, area_pixel_m2, "Mandaguari")
results_mari = calculate_areas(gdf_mari, area_pixel_m2, "Marialva")
results_rol = calculate_areas(gdf_rol, area_pixel_m2, "Rolandia")
results_sar = calculate_areas(gdf_sar, area_pixel_m2, "Sarandi")
results_maring = calculate_areas(gdf_maring, area_pixel_m2, "Maringa")


# Calcular totais
class_list = ["bare", "bush", "crop", "grass", "hduf", "industrial", "lduf", "mduf", "tree", "water"]
total_londrina = sum([results_lda[f"area_{cls}_km2_Londrina"] for cls in class_list])
total_cambe = sum([results_camb[f"area_{cls}_km2_Cambe"] for cls in class_list])
total_ibipora = sum([results_ibip[f"area_{cls}_km2_Ibipora"] for cls in class_list])
total_apucarana = sum([results_apuc[f"area_{cls}_km2_Apucarana"] for cls in class_list])
total_arapongas = sum([results_arap[f"area_{cls}_km2_Arapongas"] for cls in class_list])
total_cambira = sum([results_cambir[f"area_{cls}_km2_Cambira"] for cls in class_list])
total_jandaia = sum([results_jand[f"area_{cls}_km2_Jandaia"] for cls in class_list])
total_mandaguari = sum([results_mand[f"area_{cls}_km2_Mandaguari"] for cls in class_list])
total_marialva = sum([results_mari[f"area_{cls}_km2_Marialva"] for cls in class_list])
total_rolandia = sum([results_rol[f"area_{cls}_km2_Rolandia"] for cls in class_list])
total_sarandi = sum([results_sar[f"area_{cls}_km2_Sarandi"] for cls in class_list])
total_maringa = sum([results_maring[f"area_{cls}_km2_Maringa"] for cls in class_list])


# Criar e exibir tabela de áreas
class_labels = {"hduf": "Developed: High Density", "mduf": "Developed: Medium Density",
                "lduf": "Developed: Low Density", "industrial": "Developed: Industrial Areas",
                "bare": "Bare Soil", "grass": "Grass/Pasture", "bush": "Shrub/Scrubs",
                "tree": "Perennial Trees", "water": "Perennial Water", "crop": "Row Crops"}

table_data = []
for class_key, class_label in class_labels.items():
    lda_km2 = results_lda[f"area_{class_key}_km2_Londrina"]
    lda_pct = (lda_km2 / total_londrina) * 100
    camb_km2 = results_camb[f"area_{class_key}_km2_Cambe"]
    camb_pct = (camb_km2 / total_cambe) * 100
    ibip_km2 = results_ibip[f"area_{class_key}_km2_Ibipora"]
    ibip_pct = (ibip_km2 / total_ibipora) * 100
    apuc_km2 = results_apuc[f"area_{class_key}_km2_Apucarana"]
    apuc_pct = (apuc_km2 / total_apucarana) * 100
    arap_km2 = results_arap[f"area_{class_key}_km2_Arapongas"]
    arap_pct = (arap_km2 / total_arapongas) * 100
    cambir_km2 = results_cambir[f"area_{class_key}_km2_Cambira"]
    cambir_pct = (cambir_km2 / total_cambira) * 100
    jand_km2 = results_jand[f"area_{class_key}_km2_Jandaia"]
    jand_pct = (jand_km2 / total_jandaia) * 100
    mand_km2 = results_mand[f"area_{class_key}_km2_Mandaguari"]
    mand_pct = (mand_km2 / total_mandaguari) * 100
    mari_km2 = results_mari[f"area_{class_key}_km2_Marialva"]
    mari_pct = (mari_km2 / total_marialva) * 100
    rol_km2 = results_rol[f"area_{class_key}_km2_Rolandia"]
    rol_pct = (rol_km2 / total_rolandia) * 100
    sar_km2 = results_sar[f"area_{class_key}_km2_Sarandi"]
    sar_pct = (sar_km2 / total_sarandi) * 100
    maring_km2 = results_maring[f"area_{class_key}_km2_Maringa"]
    maring_pct = (maring_km2 / total_maringa) * 100

    table_data.append({"Land Cover Class": class_label,
                      "Londrina": f"{lda_km2:.2f} ({lda_pct:.2f}%)",
                      "Cambé": f"{camb_km2:.2f} ({camb_pct:.2f}%)",
                      "Ibiporã": f"{ibip_km2:.2f} ({ibip_pct:.2f}%)",
                      "Apucarana": f"{apuc_km2:.2f} ({apuc_pct:.2f}%)",
                      "Arapongas": f"{arap_km2:.2f} ({arap_pct:.2f}%)",
                      "Cambira": f"{cambir_km2:.2f} ({cambir_pct:.2f}%)",
                      "Jandaia do Sul": f"{jand_km2:.2f} ({jand_pct:.2f}%)",
                      "Mandaguari": f"{mand_km2:.2f} ({mand_pct:.2f}%)",
                      "Marialva": f"{mari_km2:.2f} ({mari_pct:.2f}%)",
                      "Rolândia": f"{rol_km2:.2f} ({rol_pct:.2f}%)",
                      "Sarandi": f"{sar_km2:.2f} ({sar_pct:.2f}%)",
                      "Maringá": f"{maring_km2:.2f} ({maring_pct:.2f}%)"})

table_data.append({"Land Cover Class": "Totals (km²)",
                  "Londrina": f"{total_londrina:.2f}",
                  "Cambé": f"{total_cambe:.2f}",
                  "Ibiporã": f"{total_ibipora:.2f}",
                  "Apucarana": f"{total_apucarana:.2f}",
                  "Arapongas": f"{total_arapongas:.2f}",
                  "Cambira": f"{total_cambira:.2f}",
                  "Jandaia do Sul": f"{total_jandaia:.2f}",
                  "Mandaguari": f"{total_mandaguari:.2f}",
                  "Marialva": f"{total_marialva:.2f}",
                  "Rolândia": f"{total_rolandia:.2f}",
                  "Sarandi": f"{total_sarandi:.2f}",
                  "Maringá": f"{total_maringa:.2f}"})                      


df_table = pd.DataFrame(table_data)
print("\n" + "="*100)
print("LAND COVER AREAS BY MUNICIPALITY (DINOv2 LoRA Model)")
print("="*100)
print(df_table.to_string(index=False))
print("="*100 + "\n")

# Concatenar dataframes de uso do solo
df_uso = pd.concat([df_uso_lda, df_uso_ibip, df_uso_camb, df_uso_apuc, df_uso_arap, 
                   df_uso_cambir, df_uso_jand, df_uso_mand, df_uso_mari, 
                   df_uso_rol, df_uso_sar, df_uso_maring], ignore_index=True)
print(f"✓ Total de uso do solo combinado: {len(df_uso):,} patches\n")





# ============================================================================
# 4. PREPARAR CAMADA 1: USO DO SOLO
# ============================================================================

print("\nPreparando camada de uso do solo...")

features_uso = []
for idx, row in df_uso.iterrows():
    if idx % 1000 == 0:
        print(f"  Processando patch {idx+1}/{len(df_uso)}")
    
    polygon_coords = create_patch_polygon(row)
    predicted_class = row['predicted_class']
    color = class_colors.get(predicted_class, '#808080')  # cinza como fallback
    confidence = row['prediction_confidence']
    
    # Converter coordenadas [lat,lon] para [lon,lat] (padrão GeoJSON)
    geojson_coords = [[[coord[1], coord[0]] for coord in polygon_coords]]
    
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Polygon",
            "coordinates": geojson_coords
        },
        "properties": {
            "fillColor": color,
            "color": color,
            "weight": 0,
            "fillOpacity": 0.6,
            "class": predicted_class,
            "confidence": f"{confidence:.2%}",
            "popup": f"Land Use: {predicted_class}<br>Confidence: {confidence:.2%}"
        }
    }
    features_uso.append(feature)

geojson_uso = {
    "type": "FeatureCollection",
    "features": features_uso
}

print(f"✓ Criados {len(features_uso):,} polígonos de uso do solo")

# ============================================================================
# 5. CRIAR MAPA INTERATIVO
# ============================================================================

print("\nCriando mapa interativo...")

# Calcular centro do mapa
center_lat = df_uso['center'].apply(lambda x: parse_coordinates(x)[0]).mean()
center_lon = df_uso['center'].apply(lambda x: parse_coordinates(x)[1]).mean()

# Criar mapa base com satélite
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=12,
    tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
    attr='Google Satellite'
)

# Adicionar camada de uso do solo
layer_uso = folium.FeatureGroup(name='Land Use Classes', show=True)
folium.GeoJson(
    geojson_uso,
    style_function=lambda feature: {
        'fillColor': feature['properties']['fillColor'],
        'color': feature['properties']['color'],
        'weight': feature['properties']['weight'],
        'fillOpacity': feature['properties']['fillOpacity'],
        'opacity': 0
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['popup'],
        aliases=[''],
        labels=False
    )
).add_to(layer_uso)
layer_uso.add_to(m)

# Adicionar controle de camadas
# folium.LayerControl(position='topleft', collapsed=False).add_to(m)

print("✓ Camadas adicionadas ao mapa")

# ============================================================================
# 6. CRIAR LEGENDAS
# ============================================================================

print("Criando legendas...")

# ============================================================================
# 6. CRIAR LEGENDAS
# ============================================================================

print("Criando legendas...")

legend_uso_html = """
<div style="position: fixed; 
           bottom: 10px; left: 10px; 
           background-color: white; border:2px solid grey; z-index:9999; 
           font-size:11px; padding: 15px; max-height: 90vh; max-width: 50vw;
           overflow-y: auto; overflow-x: auto;">
<p style="margin:0 0 10px 0; font-size:13px;"><b>Land Cover Areas by Municipality (DINOv2 LoRA)</b></p>
<table style="border-collapse: collapse; font-size:8px; width:100%;">
<thead>
    <tr style="background-color: #f0f0f0;">
        <th style="border: 1px solid #999; padding: 5px; text-align: left; font-weight: bold;">Land Cover Class</th>
        <th style="border: 1px solid #999; padding: 5px; text-align: center; font-weight: bold;">Londrina<br>km² (%)</th>
        <th style="border: 1px solid #999; padding: 5px; text-align: center; font-weight: bold;">Cambé<br>km² (%)</th>
        <th style="border: 1px solid #999; padding: 5px; text-align: center; font-weight: bold;">Ibiporã<br>km² (%)</th>
        <th style="border: 1px solid #999; padding: 5px; text-align: center; font-weight: bold;">Apucarana<br>km² (%)</th>
        <th style="border: 1px solid #999; padding: 5px; text-align: center; font-weight: bold;">Arapongas<br>km² (%)</th>
        <th style="border: 1px solid #999; padding: 5px; text-align: center; font-weight: bold;">Cambira<br>km² (%)</th>
        <th style="border: 1px solid #999; padding: 5px; text-align: center; font-weight: bold;">Jandaia<br>km² (%)</th>
        <th style="border: 1px solid #999; padding: 5px; text-align: center; font-weight: bold;">Mandaguari<br>km² (%)</th>
        <th style="border: 1px solid #999; padding: 5px; text-align: center; font-weight: bold;">Marialva<br>km² (%)</th>
        <th style="border: 1px solid #999; padding: 5px; text-align: center; font-weight: bold;">Rolândia<br>km² (%)</th>
        <th style="border: 1px solid #999; padding: 5px; text-align: center; font-weight: bold;">Sarandi<br>km² (%)</th>
        <th style="border: 1px solid #999; padding: 5px; text-align: center; font-weight: bold;">Maringá<br>km² (%)</th>
    </tr>
</thead>
<tbody>
"""

# Mapeamento de classes para labels (mantendo a ordem do código original)
table_classes = [
    ('hduf', 'Developed: High Density'),
    ('mduf', 'Developed: Medium Density'),
    ('lduf', 'Developed: Low Density'),
    ('industrial', 'Developed: Industrial Areas'),
    ('bare', 'Bare Soil'),
    ('grass', 'Grass/Pasture'),
    ('bush', 'Shrub/Scrubs'),
    ('tree', 'Perennial Trees'),
    ('water', 'Perennial Water'),
    ('crop', 'Row Crops')
]

# Adicionar linhas de dados usando os cálculos já realizados
for class_key, class_label in table_classes:
    # Obter áreas calculadas dinamicamente
    lda_km2 = results_lda[f"area_{class_key}_km2_Londrina"]
    lda_pct = (lda_km2 / total_londrina) * 100
    camb_km2 = results_camb[f"area_{class_key}_km2_Cambe"]
    camb_pct = (camb_km2 / total_cambe) * 100
    ibip_km2 = results_ibip[f"area_{class_key}_km2_Ibipora"]
    ibip_pct = (ibip_km2 / total_ibipora) * 100
    apuc_km2 = results_apuc[f"area_{class_key}_km2_Apucarana"]
    apuc_pct = (apuc_km2 / total_apucarana) * 100
    arap_km2 = results_arap[f"area_{class_key}_km2_Arapongas"]
    arap_pct = (arap_km2 / total_arapongas) * 100
    cambir_km2 = results_cambir[f"area_{class_key}_km2_Cambira"]
    cambir_pct = (cambir_km2 / total_cambira) * 100
    jand_km2 = results_jand[f"area_{class_key}_km2_Jandaia"]
    jand_pct = (jand_km2 / total_jandaia) * 100
    mand_km2 = results_mand[f"area_{class_key}_km2_Mandaguari"]
    mand_pct = (mand_km2 / total_mandaguari) * 100
    mari_km2 = results_mari[f"area_{class_key}_km2_Marialva"]
    mari_pct = (mari_km2 / total_marialva) * 100
    rol_km2 = results_rol[f"area_{class_key}_km2_Rolandia"]
    rol_pct = (rol_km2 / total_rolandia) * 100
    sar_km2 = results_sar[f"area_{class_key}_km2_Sarandi"]
    sar_pct = (sar_km2 / total_sarandi) * 100
    maring_km2 = results_maring[f"area_{class_key}_km2_Maringa"]
    maring_pct = (maring_km2 / total_maringa) * 100
    
    color = class_colors[class_key]
    text_color = 'white' if class_key in ['hduf', 'mduf', 'tree'] else 'black'
    
    legend_uso_html += f"""
    <tr>
        <td style="border: 1px solid #999; padding: 5px; background-color: {color}; color: {text_color}; font-weight: bold;">{class_label}</td>
        <td style="border: 1px solid #999; padding: 5px; text-align: center;">{lda_km2:.2f} ({lda_pct:.2f}%)</td>
        <td style="border: 1px solid #999; padding: 5px; text-align: center;">{camb_km2:.2f} ({camb_pct:.2f}%)</td>
        <td style="border: 1px solid #999; padding: 5px; text-align: center;">{ibip_km2:.2f} ({ibip_pct:.2f}%)</td>
        <td style="border: 1px solid #999; padding: 5px; text-align: center;">{apuc_km2:.2f} ({apuc_pct:.2f}%)</td>
        <td style="border: 1px solid #999; padding: 5px; text-align: center;">{arap_km2:.2f} ({arap_pct:.2f}%)</td>
        <td style="border: 1px solid #999; padding: 5px; text-align: center;">{cambir_km2:.2f} ({cambir_pct:.2f}%)</td>
        <td style="border: 1px solid #999; padding: 5px; text-align: center;">{jand_km2:.2f} ({jand_pct:.2f}%)</td>
        <td style="border: 1px solid #999; padding: 5px; text-align: center;">{mand_km2:.2f} ({mand_pct:.2f}%)</td>
        <td style="border: 1px solid #999; padding: 5px; text-align: center;">{mari_km2:.2f} ({mari_pct:.2f}%)</td>
        <td style="border: 1px solid #999; padding: 5px; text-align: center;">{rol_km2:.2f} ({rol_pct:.2f}%)</td>
        <td style="border: 1px solid #999; padding: 5px; text-align: center;">{sar_km2:.2f} ({sar_pct:.2f}%)</td>
        <td style="border: 1px solid #999; padding: 5px; text-align: center;">{maring_km2:.2f} ({maring_pct:.2f}%)</td>
    </tr>
    """

# Calcular totais de áreas desenvolvidas dinamicamente
# total_dev_lda = (results_lda['area_hduf_km2_Londrina'] + results_lda['area_mduf_km2_Londrina'] + 
#                  results_lda['area_lduf_km2_Londrina'] + results_lda['area_industrial_km2_Londrina'])
# total_dev_camb = (results_camb['area_hduf_km2_Cambe'] + results_camb['area_mduf_km2_Cambe'] + 
#                   results_camb['area_lduf_km2_Cambe'] + results_camb['area_industrial_km2_Cambe'])
# total_dev_ibip = (results_ibip['area_hduf_km2_Ibipora'] + results_ibip['area_mduf_km2_Ibipora'] + 
#                   results_ibip['area_lduf_km2_Ibipora'] + results_ibip['area_industrial_km2_Ibipora'])
# total_dev_apuc = (results_apuc['area_hduf_km2_Apucarana'] + results_apuc['area_mduf_km2_Apucarana'] + 
#                   results_apuc['area_lduf_km2_Apucarana'] + results_apuc['area_industrial_km2_Apucarana'])
# total_dev_arap = (results_arap['area_hduf_km2_Arapongas'] + results_arap['area_mduf_km2_Arapongas'] + 
#                   results_arap['area_lduf_km2_Arapongas'] + results_arap['area_industrial_km2_Arapongas'])
# total_dev_cambir = (results_cambir['area_hduf_km2_Cambira'] + results_cambir['area_mduf_km2_Cambira'] + 
#                     results_cambir['area_lduf_km2_Cambira'] + results_cambir['area_industrial_km2_Cambira'])
# total_dev_jand = (results_jand['area_hduf_km2_Jandaia'] + results_jand['area_mduf_km2_Jandaia'] + 
#                   results_jand['area_lduf_km2_Jandaia'] + results_jand['area_industrial_km2_Jandaia'])
# total_dev_mand = (results_mand['area_hduf_km2_Mandaguari'] + results_mand['area_mduf_km2_Mandaguari'] + 
#                   results_mand['area_lduf_km2_Mandaguari'] + results_mand['area_industrial_km2_Mandaguari'])
# total_dev_mari = (results_mari['area_hduf_km2_Marialva'] + results_mari['area_mduf_km2_Marialva'] + 
#                   results_mari['area_lduf_km2_Marialva'] + results_mari['area_industrial_km2_Marialva'])
# total_dev_rol = (results_rol['area_hduf_km2_Rolandia'] + results_rol['area_mduf_km2_Rolandia'] + 
#                  results_rol['area_lduf_km2_Rolandia'] + results_rol['area_industrial_km2_Rolandia'])
# total_dev_sar = (results_sar['area_hduf_km2_Sarandi'] + results_sar['area_mduf_km2_Sarandi'] + 
#                  results_sar['area_lduf_km2_Sarandi'] + results_sar['area_industrial_km2_Sarandi'])
# total_dev_maring = (results_maring['area_hduf_km2_Maringa'] + results_maring['area_mduf_km2_Maringa'] + 
#                     results_maring['area_lduf_km2_Maringa'] + results_maring['area_industrial_km2_Maringa'])

legend_uso_html += f"""
    <tr style="background-color: #f0f0f0; font-weight: bold;">
        <td style="border: 1px solid #999; padding: 6px;">Total Municipality</td>
        <td style="border: 1px solid #999; padding: 6px; text-align: center;">{total_londrina:.2f} </td>
        <td style="border: 1px solid #999; padding: 6px; text-align: center;">{total_cambe:.2f}</td>
        <td style="border: 1px solid #999; padding: 6px; text-align: center;">{total_ibipora:.2f}</td>
        <td style="border: 1px solid #999; padding: 6px; text-align: center;">{total_apucarana:.2f}</td>
        <td style="border: 1px solid #999; padding: 6px; text-align: center;">{total_arapongas:.2f}</td>
        <td style="border: 1px solid #999; padding: 6px; text-align: center;">{total_cambira:.2f}</td>
        <td style="border: 1px solid #999; padding: 6px; text-align: center;">{total_jandaia:.2f}</td>
        <td style="border: 1px solid #999; padding: 6px; text-align: center;">{total_mandaguari:.2f}</td>
        <td style="border: 1px solid #999; padding: 6px; text-align: center;">{total_marialva:.2f}</td>
        <td style="border: 1px solid #999; padding: 6px; text-align: center;">{total_rolandia:.2f}</td>
        <td style="border: 1px solid #999; padding: 6px; text-align: center;">{total_sarandi:.2f}</td>
        <td style="border: 1px solid #999; padding: 6px; text-align: center;">{total_maringa:.2f}</td>
    </tr>
</tbody>
</table>
</div>
"""

# Adicionar legendas ao mapa
m.get_root().html.add_child(folium.Element(legend_uso_html))

print("✓ Legendas criadas")

# ============================================================================
# 8. SALVAR MAPA
# ============================================================================


output_file = '/Users/fjcosta/Documents/landCoverlandValue/landcover/landcoverDINO_NP.html'
m.save(output_file)

print(f"\n{'='*70}")
print(f"✅ MAPA CRIADO COM SUCESSO!")
print(f"{'='*70}")
# print(f"Arquivo: {output_file}")
print(f"\nCamadas:")
print(f"  • Land Use Classes: {len(features_uso):,} patches")
print(f"\nUse o controle de camadas no mapa para ligar/desligar cada camada.")
print(f"{'='*70}")

Carregando dados...
✓ Uso do solo: 18,302 patches
✓ Uso do solo: 3,903 patches
✓ Uso do solo: 5,905 patches
✓ Uso do solo: 9,043 patches
✓ Uso do solo: 10,671 patches
✓ Uso do solo: 238 patches
✓ Uso do solo: 1,693 patches
✓ Uso do solo: 3,520 patches
✓ Uso do solo: 2,297 patches
✓ Uso do solo: 3,771 patches
✓ Uso do solo: 2,342 patches
✓ Uso do solo: 11,481 patches

Processando dados para cálculo de áreas...

LAND COVER AREAS BY MUNICIPALITY (DINOv2 LoRA Model)
           Land Cover Class       Londrina          Cambé        Ibiporã      Apucarana      Arapongas       Cambira Jandaia do Sul     Mandaguari       Marialva       Rolândia        Sarandi        Maringá
    Developed: High Density   4.43 (2.03%)   0.01 (0.02%)   0.01 (0.03%)   0.36 (0.33%)   0.11 (0.08%)  0.00 (0.00%)   0.05 (0.24%)   0.00 (0.00%)   0.00 (0.00%)   0.01 (0.03%)   0.01 (0.04%)   1.23 (0.90%)
  Developed: Medium Density  16.90 (7.73%)   2.45 (3.47%)   0.75 (1.61%)   4.04 (3.74%)   2.86 (2.24%)  0.06 (2.10%)   